## Como ganhar uma partida de League of Legends com estatística.

Neste notebook iremos fazer uma análise exploratória dos dados dos melhores jogadores de Lol de todo o mundo e entender quais são os fatores que levam uma partida à vitória ou derrota.

Nosso primeiro passo será coletar os dados das partidas na API da Riot Games, todos os dados são públicos e para fazer requisições é necessário um token de acesso à API. Há 16 servidores ativos de League of Legends em todo o mundo, sendo dividido em 4 regiões: Américas, Europa, Ásia e Oceania.

Estaremos analisando as últimas 100 partidas dos melhores jogadores de todos os servidores. Mas o que define quem são os melhores jogadores? Além do fato de alguns destes serem jogadores profissionais, os melhores jogadores estão no maior ranking do jogo: Challenger. Cada servidor possui entre 100 e 200 Challengers(cerca de 0.01% dos jogadores)

In [1]:
import requests
import pandas as pd
import time
import os
from dotenv import load_dotenv

Para fazer requisições para a  API da Riot Games, você precisa de um token de acesso. O token pode ser solicitado no link: https://developer.riotgames.com/

In [2]:
load_dotenv()
RGKEY = os.getenv("RGKEY")

Nosso primeiro passo é fazer a requisição dos ID's de todos os jogadores Challenger de todos os servidores do mundo. Para isso iremo definir um dicionário definindo a região de cada servidor, depois iremos iterar sobre esse dicionário e fazer os requests na rota **league-exp** usando o **servidor** como parâmetro.

In [11]:
regions = {
    'BR1': "AMERICAS",
    'EUN1': "EUROPE",
    'EUW1': "EUROPE",
    'JP1': "ASIA",
    'KR': "ASIA",
    'LA1': "AMERICAS",
    'LA2': "AMERICAS",
    'NA1': "AMERICAS",
    'OC1': "SEA",
    'PH2': "SEA",
    'RU': "EUROPE",
    'SG2': "SEA",
    'TH2': "SEA",
    'TR1': "EUROPE",
    'TW2': "SEA",
    'VN2': "SEA"
  }

In [12]:
summoners = []
for server, region in regions.items():
  URL = f"https://{server.lower()}.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1&api_key={RGKEY}"
  response = requests.get(URL).json()
  for player in response:
    summoners.append({"summonerId": player["summonerId"], "region": region, "server": server})

In [15]:
summoners = pd.DataFrame(summoners).sample(300)

Para fazer a requisição das partidas de cada jogador, precisamos do **puuid**, que pode ser puxado através do **summoneId** que coletamos acima. Para isso iremos fazer requests na rota **summoner** e utilizar o **summonerId** e o **servidor** como parâmetros.

In [16]:
for i, row in summoners.iterrows():
  time.sleep(1)
  summoner_id = row['summonerId']
  server = row['server']

  URL = f"https://{server}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={RGKEY}"
  response = requests.get(URL).json()

  summoners.loc[i, 'puuid'] = response['puuid']

In [17]:
summoners

,summonerId,region,server,puuid
662,c9Nba54KkFQ6NI3qL9gY45xgMk7XSlg6AWWWcBaISzjPPq...,SEA,VN2,FYG7pdocudpaUZQMmAvavzbQXsIXGqPXBJsKlBsp6Ttgp7...


Agora nós temos o **puuid** de dos melhores jogadores do mundo. Nosso próximo passo será consultar as últimas 100 partidas de todos os jogadores. Cada partida tem um ID, vamos fazer a requisição dos **match_ids** utilizando a rota **match** utilizando o **puuid** e a **região** como parâmetros.

In [18]:
matchs = []
for i, row in summoners.iterrows():
  time.sleep(1)
  puuid = row['puuid']
  region = row['region']

  URL = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=100&api_key={RGKEY}"
  try:
    response = requests.get(URL).json()
    for id in response:
      matchs.append({'match_id': id, 'region': region})
  except:
    pass

In [19]:
matchs = pd.DataFrame(matchs)

In [20]:
matchs

,match_id,region
0,VN2_449182594,SEA
1,VN2_449156174,SEA
2,VN2_449118838,SEA
3,VN2_449096415,SEA
4,VN2_449047525,SEA
...,...,...
95,VN2_436545044,SEA
96,VN2_436498390,SEA
97,VN2_436457832,SEA
98,VN2_436436219,SEA


### Coletando os dados das partidas.

Agora chegamos no nosso último passo, que é coletar todos os dados de todas as partidas. Através da API da Riot, conseguimos informações como a rota em que o jogador jogou, quantidade de abates e mortes, quantidade de sentinelas e pings utilizados, quais itens foram comprados, com qual campeão ele estava jogando, etc...

Com todas essas informações, além de entender quais são os fatores que levam um jogador ao topo, poderemos analisar também quais são as melhores composições para times(não se esqueça que o lol é um jogo cooperativo), poderemos entender a melhor forma de construir uma composição que faça sentido para o time, combinações de campeões e itens, etc...

Para fazer a coleta dos dados, iremos utilizar a rota **match** novamente e passar o **match_id** como parâmetro. Também iremos criar um dataframe para cada rota, definidas na lista **positions**

In [21]:
tops, jgs, mids, adcs, sups = [], [], [], [], []
columns_to_remove = ['challenges', 'missions', 'perks']
setter = {'TOP': tops, 'JUNGLE': jgs, 'MIDDLE': mids, 'BOTTOM': adcs, 'UTILITY': sups}
positions = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']

In [22]:
for i, row in matchs.iterrows():
  time.sleep(1)
  match_id = row['match_id']
  region = row['region']

  URL = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={RGKEY}"
  try:
    response = requests.get(URL).json()
    participants = response['info']['participants']
    for participant in participants:
      for column in columns_to_remove:
        del participant[column]
      for position in positions:
        if participant['teamPosition'] == position:
          setter.get(position).append(participant)
  except:
    pass


In [ ]:
for role, dataframe in setter.items():
  pd.DataFrame(dataframe).to_excel(f'original/{role}.xlsx')